Pontifícia Universidade Católica de São Paulo 

`Ciência de Dados e Inteligência Artificial`

🎓 Laboratório 7 - Classificador Naive Bayes (Parte 2)

---
> 👨‍🏫*Professor Rooney Coelho (rracoelho@pucsp.br)*
---


In [4]:
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
import string

import pandas as pd
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import naive_bayes

Carregue o dataset de SPAM para SMS

In [5]:
df = pd.read_csv('sms_spam.csv', encoding='latin1')
df = df[['v1','v2']]
df.columns = ['categoria', 'mensagem']
df

categoria                                           mensagem
0          ham  Go until jurong point, crazy.. Available only ...
1          ham                      Ok lar... Joking wif u oni...
2         spam  Free entry in 2 a wkly comp to win FA Cup fina...
3          ham  U dun say so early hor... U c already then say...
4          ham  Nah I don't think he goes to usf, he lives aro...
...        ...                                                ...
5567      spam  This is the 2nd time we have tried 2 contact u...
5568       ham              Will Ì_ b going to esplanade fr home?
5569       ham  Pity, * was in mood for that. So...any other s...
5570       ham  The guy did some bitching but I acted like i'd...
5571       ham                         Rofl. Its true to its name

[5572 rows x 2 columns]

### Pré-processamento

Traduza a coluna `categoria` para `ham: 0` e `spam: 1` use o método do Pandas adequado para isto.

In [6]:
df_pproc = df.copy()
df_pproc['categoria'] = df_pproc['categoria'].map({'ham':0, 'spam':1})

Use a função abaixo para criar uma nova coluna no dataset para o texto pré-processado.

1. Remoção de pontuação
2. Passar o texto para caixa baixa
3. Remover palavras vazias
4. Remover dígitos

In [7]:
def preprocessText(msg):
    removepunc = [c for c in msg if c not in string.punctuation]
    removepunc = "".join(removepunc)
    lower = removepunc.lower()
    tokenize = nltk.word_tokenize(lower)
    remove_stop = [word for word in tokenize if word not in stopwords.words('english')]
    remove_digit = [word for word in remove_stop if not word.isdigit()]
    joinagain = " ".join(remove_digit)
    return joinagain

Crie a coluna `mensagem_pre` e use o método `apply()` do Pandas para varrear o Dataframe com a função definida.

In [13]:
nltk.download('punkt')
nltk.download('stopwords')
df_pproc['mensagem_pre'] = df_pproc['mensagem'].apply(preprocessText)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ISAAC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ISAAC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [14]:
df_pproc

categoria                                           mensagem  \
0             0  Go until jurong point, crazy.. Available only ...   
1             0                      Ok lar... Joking wif u oni...   
2             1  Free entry in 2 a wkly comp to win FA Cup fina...   
3             0  U dun say so early hor... U c already then say...   
4             0  Nah I don't think he goes to usf, he lives aro...   
...         ...                                                ...   
5567          1  This is the 2nd time we have tried 2 contact u...   
5568          0              Will Ì_ b going to esplanade fr home?   
5569          0  Pity, * was in mood for that. So...any other s...   
5570          0  The guy did some bitching but I acted like i'd...   
5571          0                         Rofl. Its true to its name   

                                           mensagem_pre  
0     go jurong point crazy available bugis n great ...  
1                               ok lar joking wif u oni  
2     free entry wkly comp win fa cup final tkts 21s...  
3                   u dun say early hor u c already say  
4           nah dont think goes usf lives around though  
...                                                 ...  
5567  2nd time tried contact u u å£750 pound prize c...  
5568                        ì b going esplanade fr home  
5569                        pity mood soany suggestions  
5570  guy bitching acted like id interested buying s...  
5571                                     rofl true name  

[5572 rows x 3 columns]

### Extração de Features

Importe o `CountVectorizer` do Scikit-Learn

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

Inicialize o Bag of Words para as mensagens pré-processadas.

Dica:
```python
vectorizer = CountVectorizer()
vectorizer.fit(df[coluna].values)
```

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(df_pproc['mensagem_pre'].values)

CountVectorizer()

Crie as features a serem usadas na classificação, armazenadas em `X`.

Dica:
```python
X = vectorizer.transform(df[coluna].values)
```

In [18]:
X = vectorizer.transform(df_pproc['mensagem_pre'].values)

Chame o target, que é a categoria, de `y`. 

In [19]:
y = df_pproc['categoria'].values

### Segmentação dos dados

Realize a segmentação dos dados usando o `train_test_split`. Separe 20% dos dados para teste.

In [20]:
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.2, random_state=42)

### Treinamento do classificador

Treine o classificador Multinomial Naive Bayes com a partição feita.

In [21]:
mnb = naive_bayes.MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

Avalie a precisão de seu classificador usando o `score`.

In [22]:
mnb.score(X_test, y_test)

0.97083239847431

### Avalie agora o seu modelo nas seguintes mensagens:

In [31]:
msg1 = 'Congratulations! You`ve won a $1,000 Walmart gift card. Go to http://bit.ly/123456 to claim now.'
msg2 = 'Hello son, when are you coming back home? Mommy'

* Não se esqueça de fazer o pré-processamento para inserir no modelo.

dica:
```python
vectorizer.transform([texto_preprocessado])
```

In [33]:
sms1 = preprocessText(msg1)
sms2 = preprocessText(msg2)

In [34]:
X2, y2 = vectorizer.transform([sms1, sms2])

In [35]:
mnb.predict(X2)

array([1], dtype=int64)

In [36]:
mnb.predict(y2)

array([0], dtype=int64)